# Basic work

### Setup

In [1]:
var moment = require('moment');
var { constants } = require('../utils');
var { advanceTimeAndBlock } = require('../utils/evm');
var { toUnit } = require('../utils/bn');
var { getLatestBlockTimestamp, getBlockTimestamp } = require('../utils/evm');
var { next, clear } = require('../utils/jupyter');
var { Notebook } = require('../utils/notebook');

clear();

In [2]:
var notebook = new Notebook();

In [3]:
next(async () => {
    await notebook.setup();
});

### Add LP tokens to job

In [4]:
next(async () => {
    await notebook.addLiquidityToJob(toUnit(1));
});

### Work, work, work
Sleep 72 hours, then start working every 72 hours, for a total of 4 periods

In [5]:
next(async () => {    
    const timeToWork = notebook.rewardPeriod * 4;
    const startedToWorkAt = await getLatestBlockTimestamp();
    const sleepTime = moment.duration(72, 'hours').as('seconds');
    
    console.log('Start of simulation');
    await notebook.recordCredits();
    
    // sleep
    await advanceTimeAndBlock(sleepTime);
    await notebook.recordCredits();
    
    // until 4 periods have not passed
    while ((await getLatestBlockTimestamp()) - startedToWorkAt < timeToWork) {
        // work
        await notebook.job.connect(notebook.keeper).work();
        await notebook.recordCredits();
        
        // sleep
        await notebook.sleepAndRecord(sleepTime, moment.duration(4, 'hours').as('seconds'));
        await notebook.recordCredits();
    }
    
    console.log('End of simulation');
});

In [6]:
next(async () => {
    await notebook.draw();
});

Start of simulation
End of simulation
